# Clasificación de IRIS con k-vecinos más cercanos (kNN)

## Objetivo
Aplicar el algoritmo de aprendizaje supervisado k-vecinos más cercanos (kNN) al conjunto de datos IRIS, desarrollando un proceso completo de clasificación que incluya: análisis exploratorio, preparación de datos, entrenamiento, evaluación del modelo y visualización de resultados.

## Cargamos la base de datos de IRIS

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# crear directorio para guardar figuras
os.makedirs('out', exist_ok=True)

# cargar el dataset IRIS
iris = pd.read_csv('iris.csv', header=None)
iris.columns = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm', 'Species']

# agregar columna Id como en el dataset original
iris.insert(0, 'Id', range(1, len(iris) + 1))

## Análisis Exploratorio de Datos

In [2]:
# Mostramos las primeras 5 filas de la base de datos
iris.head()

,5.1,3.5,1.4,0.2,Iris-setosa
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [3]:
# Mostramos las últimas 5 filas de la base de datos
iris.tail()

,5.1,3.5,1.4,0.2,Iris-setosa
144,6.7,3.0,5.2,2.3,Iris-virginica
145,6.3,2.5,5.0,1.9,Iris-virginica
146,6.5,3.0,5.2,2.0,Iris-virginica
147,6.2,3.4,5.4,2.3,Iris-virginica
148,5.9,3.0,5.1,1.8,Iris-virginica


In [4]:
# Mostramos la información de la base de datos
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   5.1          149 non-null    float64
 1   3.5          149 non-null    float64
 2   1.4          149 non-null    float64
 3   0.2          149 non-null    float64
 4   Iris-setosa  149 non-null    object 
dtypes: float64(4), object(1)
memory usage: 5.9+ KB


In [5]:
# Mostramos la descripción estadística de la base de datos 'iris'
iris.describe()

,5.1,3.5,1.4,0.2
count,149.000000,149.000000,149.000000,149.000000
mean,5.848322,3.051007,3.774497,1.205369
std,0.828594,0.433499,1.759651,0.761292
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.400000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


## Preprocesamiento de datos

In [6]:
# Eliminamos la columna 'Id' de la base de datos 'iris'
iris = iris.drop('Id', axis=1)
iris.head()

KeyError: "['Id'] not found in axis"

## Visualización de Datos

### Comparamos las variables 'Sepal Length' y 'Sepal Width'

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='SepalLengthCm', y='SepalWidthCm', hue='Species', data=iris)
plt.title('Relación entre Longitud y Ancho del Sépalo')
plt.savefig('out/sepal_scatter.png', dpi=300, bbox_inches='tight')
plt.show()

### Comparamos las variables 'Petal Length' y 'Petal Width'

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='PetalLengthCm', y='PetalWidthCm', hue='Species', data=iris)
plt.title('Relación entre Longitud y Ancho del Pétalo')
plt.savefig('out/petal_scatter.png', dpi=300, bbox_inches='tight')
plt.show()

### Graficamos las relaciones de todas las columnas

In [ ]:
# Visualizamos la relación entre las características
g = sns.pairplot(iris, hue='Species')
g.savefig('out/pairplot_iris.png', dpi=300, bbox_inches='tight')
plt.show()

## Modelado

### Aislamos las variables predictoras

In [ ]:
# Aislamos las variables predictoras
X = iris.drop('Species', axis=1)
X.head()

### Aislamos la variable objetivo

In [ ]:
# Aislamos la variable objetivo
y = iris['Species']
y.head()

### Dividimos los datos en sub datasets para entrenamiento y prueba

In [ ]:
# Dividimos la base de datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Entrenamiento: {len(X_train)} muestras")
print(f"Prueba: {len(X_test)} muestras")

### Entrenamos el clasificador k-NN

In [ ]:
# Importamos el modelo k-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier

# Creamos el modelo K-Nearest Neighbors
classifier = KNeighborsClassifier(n_neighbors=5)

# Entrenamos el modelo
classifier.fit(X_train, y_train)

In [ ]:
# Realizamos la predicción en los datos de X_test
y_pred = classifier.predict(X_test)
y_pred

## Evaluación del modelo

In [ ]:
# Importamos la métrica de precisión
from sklearn.metrics import accuracy_score

# Calculamos la precisión del modelo
acc = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", acc)

## Matriz de Confusión

In [ ]:
# Importamos la matriz de confusión
from sklearn.metrics import confusion_matrix

# Calculamos la matriz de confusión con los nombres de las especies
cf_matrix = confusion_matrix(y_test, y_pred)

# Visualizamos la matriz de confusión con los nombres de las especies de IRIS
plt.figure(figsize=(8, 6))
sns.heatmap(cf_matrix, annot=True, fmt='d', xticklabels=iris['Species'].unique(), yticklabels=iris['Species'].unique(), cmap='Blues')
plt.title('Matriz de Confusión - Modelo k-NN')
plt.ylabel('Valores Reales')
plt.xlabel('Predicciones')
plt.savefig('out/confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

## Experimentación con diferentes valores de k

In [ ]:
# Probamos diferentes valores de k
k_values = range(1, 16)
accuracies = []

print("Precisión por valor de k:")
for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_pred_k = knn.predict(X_test)
    acc_k = accuracy_score(y_test, y_pred_k)
    accuracies.append(acc_k)
    print(f"k={k}: {acc_k:.4f} ({acc_k*100:.1f}%)")

# Visualizamos los resultados
plt.figure(figsize=(10, 6))
plt.plot(k_values, accuracies, marker='o', linewidth=2, markersize=6)
plt.title('Precisión del modelo k-NN para diferentes valores de k')
plt.xlabel('Valor de k')
plt.ylabel('Precisión')
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.savefig('out/k_values_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# Encontramos el mejor k
best_k = k_values[accuracies.index(max(accuracies))]
best_acc = max(accuracies)
print(f"\nMejor valor de k: {best_k} con precisión de {best_acc:.4f} ({best_acc*100:.1f}%)")